<a href="https://colab.research.google.com/github/janilles/couch/blob/master/couch_runs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boilerplate

In [0]:
# added -q for suppressing output
!pip install -q -U pymysql

import pymysql
import pandas as pd
import altair as alt


# Database connection



In [0]:
def connect():
    return pymysql.connect(
        host='',
        port=,
        user='',
        passwd='',
        db='',
        autocommit=True
        )

connection = connect()

def sql_to_df(sql):
    return pd.read_sql(sql, con=connection)


# Database tables

In [0]:
tables = sql_to_df("""SHOW TABLES""")

tables


## Runs table described

In [0]:
sql_to_df("""DESCRIBE c25k_run""")

# Data sense-checking

## Run timestamps on iOS v Android
Some version of Android is believed to be an hour behind but it doesn't look like it in comparison to iOS.

In [0]:
run_times = sql_to_df("""
    SELECT
        A.device_os,
        HOUR(started_at) AS run_hour,
        COUNT(DISTINCT started_at) AS run_count,
        COUNT(DISTINCT started_at)*100.00/B.total_run_count AS pc_total_runs
    FROM
        insights.c25k_run AS A

        LEFT JOIN
            (SELECT
                device_os,
                COUNT(DISTINCT started_at) AS total_run_count
            FROM
                insights.c25k_run
            GROUP BY
                device_os) AS B
        ON A.device_os = B.device_os
        
    GROUP BY
        A.device_os,
        run_hour
    """)

run_times.head()


In [0]:
def farrukh(row):
    if row['device_os'] == 'android':
        hour = row['run_hour'] - 1
        return hour
    else:
        return row['run_hour']

run_times['shifted_hour'] = run_times.apply(
    lambda row: farrukh(row),axis=1)

run_times

In [0]:
alt.Chart(run_times,
          title='Percent of all runs by hour and OS').mark_line(
).encode(
    x='run_hour',
    y='pc_total_runs',
    color='device_os')


In [0]:
alt.Chart(run_times,
          title='Percent of all runs by hour and OS').mark_line(
).encode(
    x='shifted_hour',
    y='pc_total_runs',
    color='device_os')


## Users with too many runs started in one day

In [0]:
started_runs_by_day = sql_to_df("""
    SELECT
        device_id,
        DATE(started_at) AS date,
        COUNT(id) AS runs_started
    FROM
        c25k_run
    GROUP BY
        device_id,
        date
    ORDER BY
        runs_started DESC""")

started_runs_by_day.head(10)


### In proportion with all users running

In [0]:
# .sum() added so it returns a number
all_users_running = sql_to_df("""
    SELECT
        COUNT(DISTINCT device_id) AS runners
    FROM
        c25k_run""")['runners'].sum()

# conditions for dataframe slicing
condition_3 = started_runs_by_day['runs_started'] > 3
condition_1 = started_runs_by_day['runs_started'] == 1

# device counts based on conditions
more_than_3 = started_runs_by_day[condition_3]['device_id'].nunique()
just_1_started = started_runs_by_day[condition_1]['device_id'].nunique()

# calculations rounded to 1 decimal
percent_3 = round(more_than_3 / all_users_running * 100, 1)
percent_1 = round(just_1_started / all_users_running * 100, 1)

print(f"All users with runs in the databse: {all_users_running}")
print(f"Users with three plus started runs in one day: {more_than_3}")
print(f"Percentage of three plus from all users: {percent_3}%")
print(f"Just one started run in a day: {just_1_started}")
print(f"Percentage of just one from all users: {percent_1}%")


## Users with too many runs finished in one day

In [0]:
runs_finished_by_day = sql_to_df("""
    SELECT
        device_id,
        DATE(started_at) AS date,
        COUNT(id) AS runs_finished
    FROM
        c25k_run
    WHERE
        has_finished = 1
    GROUP BY
        device_id,
        date
    ORDER BY
        runs_finished DESC""")

runs_finished_by_day.head(10)


### In proportion with all users finishing runs

In [0]:
# .sum() added so it doesn't return a dataframe but a figure
all_users_finishing = sql_to_df("""
    SELECT
        COUNT(DISTINCT device_id) AS finishers
    FROM
        c25k_run
    WHERE
        has_finished = 1
    """)['finishers'].sum()

# conditions for dataframe slicing
condition_2f = runs_finished_by_day['runs_finished'] > 1
condition_1f = runs_finished_by_day['runs_finished'] == 1

# device counts based on conditions
finished_2plus = runs_finished_by_day[condition_2f]['device_id'].nunique()
finished_1 = runs_finished_by_day[condition_1f]['device_id'].nunique()

# calculations rounded to 1 decimal
percent_2f = round(finished_2plus / all_users_finishing * 100, 1)
percent_1f = round(finished_1 / all_users_finishing * 100, 1)

print(f"All users with runs finished: {all_users_finishing}")
print(f"Users with two or more runs finished in one day: {finished_2plus}")
print(f"Percentage of two or more... from all users: {percent_2f}%")
print(f"Just one finished run in a day: {finished_1}")
print(f"Percentage of just one from all users: {percent_1f}%")


## Users with many runs finished in a day by device OS

In [0]:
runs_finished_by_day_os = sql_to_df("""
    SELECT
        device_id,
        device_os,
        DATE(started_at) AS date,
        COUNT(id) AS runs_finished
    FROM
        c25k_run
    WHERE
        has_finished = 1
    GROUP BY
        device_id,
        date
    ORDER BY
        runs_finished DESC""")

runs_finished_by_day_os.head()


In [0]:
# just a sample of the worst cases
ten_or_more = runs_finished_by_day_os['runs_finished'] > 9

runs_finished_by_day_os.loc[ten_or_more].groupby(
    'device_os',
    as_index=False)['device_id'].count()


# Runs data overview

In [0]:
overview = sql_to_df("""
    SELECT
        FORMAT(COUNT(DISTINCT device_id), 0) AS users,
        FORMAT(COUNT(id), 0) AS runs_started,
        FORMAT(SUM(has_reached_halfway), 0) AS runs_halfway,
        FORMAT(SUM(has_finished), 0) AS runs_finished,
        FORMAT(SUM(IF(lat != 0, 1, 0))/COUNT(id)*100, 1) AS pc_runs_with_location
    FROM
        c25k_run
        """)

overview


# Runs by day

In [0]:
runs_by_date = sql_to_df("""
    SELECT
        DATE(started_at) AS date,
        DAYNAME(started_at) AS day,
        COUNT(id) AS runs_started
    FROM
        c25k_run
    WHERE
        DATE(started_at) BETWEEN '2019-04-28' AND '2019-07-03' -- data cleaning
    GROUP BY
        date
    """)

runs_by_date.head()

In [0]:
#@title Viz
# this is necessary as the 'date' is an object, not datetime
runs_by_date['date'] = pd.to_datetime(runs_by_date['date'])

# chart from Altair viz example gallery
bar = alt.Chart(runs_by_date, title='Run starts').mark_bar(size=15).encode(
    x='date:T',
    y='runs_started:Q',
    color=alt.Color('day:N', sort=['Monday',
                                   'Tuesday',
                                   'Wednesday',
                                   'Thursday',
                                   'Friday',
                                   'Saturday',
                                   'Sunday'])
)

rule = alt.Chart(runs_by_date).mark_rule(color='red').encode(
    y='mean(runs_started):Q'
)

(bar + rule).properties(width=1000, height=500).configure_axis(
    grid=False
)


# Runs by hour of day

In [0]:
# count of run IDs per weekday and hour averaged
runs_by_hour = sql_to_df("""
    SELECT
        s.day,
        s.hour,
        AVG(s.runs) AS avg_runs
    FROM
        (
        SELECT
            DAYNAME(started_at) AS day,
            HOUR(started_at) AS hour,
            COUNT(id) AS runs
        FROM
            c25k_run
        GROUP BY
            day,
            hour
        ) s
    GROUP BY
        s.day,
        s.hour
    """)

runs_by_hour.head()


In [0]:
#@title Viz
highlight = alt.selection(type='single',
                          on='mouseover',
                          fields=['day'],
                          nearest=True)

base = alt.Chart(runs_by_hour, 
                 title='Avg. number of runs by hour of day').encode(
    x='hour:N',
    y='avg_runs:Q',
    color=alt.Color('day:N', sort=['Monday',
                                   'Tuesday',
                                   'Wednesday',
                                   'Thursday',
                                   'Friday',
                                   'Saturday',
                                   'Sunday'])
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=1000, height=500
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
).properties(width=1000, height=500)

points + lines

# Runs started/completed

In [0]:
run_start_comp = sql_to_df("""
    SELECT
        ((week_no-1)*3 + run_no) AS overall_run_no,
        week_no,
        COUNT(DISTINCT device_id) AS runners,
        COUNT(DISTINCT (CASE WHEN has_finished = 1 THEN started_at END)) AS finished_run
    FROM
        c25k_run        
    GROUP BY
        overall_run_no
    """)

run_start_comp.head()


In [0]:
melted_st_comp = pd.melt(run_start_comp, id_vars=['overall_run_no', 'week_no'], value_vars=['runners', 'finished_run'])

melted_st_comp.head()

In [0]:
#@title Viz
alt.Chart(melted_st_comp, 
          title='Runners with completed runs').mark_bar(
).encode(
    # x='variable:O',
    alt.X('variable:O',
          scale=alt.Scale(rangeStep=15),
          sort=['runners', 'finished_run'],
          title=''), # to hide labels
    y='value:Q',
    color='week_no:N',
    column='overall_run_no:N'
)


# Completion rates for each run 

In [0]:
#@title Default title text
runs = sql_to_df("""
    SELECT
        id,
        week_no,
        run_no,
        has_reached_halfway,
        has_finished
    FROM
        c25k_run
    """)

runs.head()

In [0]:
#@title Default title text
# initiate lists to build a data fram from
Starts = []
Halfway = []
Finished = []

for week in range(1, 10):
    # dataframe slicing condition
    week_slice = runs['week_no'] == week
    # sliced dtaframe by week number from the for loop
    wk_runs = runs[week_slice]

    for run in range(1, 4):
        # dataframe slicing condtions
        run_slice = wk_runs['run_no'] == run
        hway_slice = wk_runs['has_reached_halfway'] == 1
        fin_slice = wk_runs['has_finished'] == 1

        # counting run IDs where conditions are met
        starts = wk_runs[run_slice]['id'].count()
        halfway = wk_runs[(run_slice) & (hway_slice)]['id'].count()
        finished = wk_runs[(run_slice) & (fin_slice)]['id'].count()

        Starts.append(starts)
        Halfway.append(halfway)
        Finished.append(finished)

# dictionary to create dataframe from
d = {'Run_no': list(range(1, 28)),
     'Starts': Starts,
     'Halfway': Halfway,
     'Finished': Finished}

run_completions = pd.DataFrame(data=d)

# create columns for each rate
run_completions['Halfway_from_start_%'] = \
    round(
        run_completions['Halfway'] /
        run_completions['Starts']*100,
        1)

run_completions['Finished_from_start_%'] = \
    round(
        run_completions['Finished'] /
        run_completions['Starts']*100,
        1)

run_completions['Finished_from_halfway_%'] = \
    round(
        run_completions['Finished'] /
        run_completions['Halfway']*100,
        1)

# add conditional formatting to relevant columns
run_completions \
    .style \
    .background_gradient(
        cmap='RdYlGn',
        subset=['Halfway_from_start_%',
                'Finished_from_start_%',
                'Finished_from_halfway_%'])


### Export dataframe as CSV

In [0]:
# from google.colab import files

# run_completions.to_csv('df.csv')
# files.download('df.csv')

# Completion rates for each run by trainer, OS

In [0]:
#@title Table
# app version not yet added
rates = sql_to_df("""
    
    SELECT
        COALESCE((week_no-1)*3 + run_no, 0) AS run_no,
        trainer_id,
        device_os,   
        COUNT(id) AS no_starts,
        COUNT(CASE WHEN has_reached_halfway = 1 THEN 1 END) AS no_halfway,
        COUNT(CASE WHEN has_finished = 1 THEN 1 END) AS no_finished,
        ROUND(SUM(has_reached_halfway)*100.0/COUNT(id),2) AS pc_halfway_start,
        ROUND(SUM(has_finished)*100.0/COUNT(id),2) AS pc_finish_start,
        ROUND(SUM(has_finished)*100.0/SUM(has_reached_halfway),2) AS pc_finish_halfway

    FROM
        c25k_run
        
    WHERE
        trainer_id != ''
    GROUP BY
        ((week_no-1)*3 + run_no),
        trainer_id, 
        device_os
    """)

rates


In [0]:
alt.Chart(rates.loc[rates['device_os'] == 'ios'],
          title='Run completion rates iOS').mark_line(
).encode(
    x='run_no',
    y='pc_finish_start',
    color='trainer_id')


In [0]:
alt.Chart(rates.loc[rates['device_os'] == 'android'],
          title='Run completion rates Android').mark_line(
).encode(
    x='run_no',
    y='pc_finish_start',
    color='trainer_id')


In [0]:
alt.Chart(rates.loc[rates['device_os'] == 'ios'],
          title='Run starts iOS').mark_line(
).encode(
    x='run_no',
    y='no_starts',
    color='trainer_id')

In [0]:
alt.Chart(rates.loc[rates['device_os'] == 'android'],
          title='Run starts Android').mark_line(
).encode(
    x='run_no',
    y='no_starts',
    color='trainer_id')


# Graduations

In [0]:
grads_all = sql_to_df("""SELECT * FROM ec_runs_no_skips""")

grads_all.head()


In [0]:
melted_all = pd.melt(grads_all, id_vars=['overall_run_no'], value_vars=['total_users', 'no_skip_users'])

melted_all.head()


In [0]:
#@title Viz
alt.Chart(melted_all).mark_bar().encode(
    #x='variable:O',
    alt.X('variable:O', 
          sort=['total_users', 'no_skip_users'],
          scale=alt.Scale(rangeStep=15), 
          # to hide labels add empty title
          title=''),
    y='value:Q',
    color='variable:N',
    column='overall_run_no:N'
)


# Graduations of cohort

In [0]:
cohort_grad = sql_to_df("""

    SELECT 
        overall_run_no,
        COUNT(*) AS total_users,
        SUM(CASE WHEN overall_run_no = run_count THEN 1 ELSE 0 END) AS no_skip_users
        
    FROM
        ec_run_stats_rank 
         
         -- restrict to users that have completed run 1 and date started
         INNER JOIN (SELECT device_id
                     FROM ec_run_stats
                     WHERE overall_run_no = 1
                     AND no_completed > 0 
                     AND first_started BETWEEN '2019-05-06' AND '2019-05-12'
                     GROUP BY device_id) AS B
         ON ec_run_stats_rank.device_id = B.device_id
    
    GROUP BY 
        overall_run_no
        """)

cohort_grad.head()

In [0]:
cohort_grad.tail()

In [0]:
melted_grad = pd.melt(cohort_grad, id_vars=['overall_run_no'], value_vars=['total_users', 'no_skip_users'])

melted_grad.head()

In [0]:
#@title Viz
alt.Chart(melted_grad).mark_bar().encode(
    #x='variable:O',
    alt.X('variable:O', 
          sort=['total_users', 'no_skip_users'],
          scale=alt.Scale(rangeStep=15), 
          # to hide labels add empty title
          title=''),
    y='value:Q',
    color='variable:N',
    column='overall_run_no:N'
)

# Completion times per run

In [0]:
# increase the max number of rows for altair plot
from altair import pipe, limit_rows, to_values
t = lambda data: pipe(data, limit_rows(max_rows=1000000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')

In [0]:
run_stats = sql_to_df("""
    SELECT
        overall_run_no,
        time_taken,
        week_no
    FROM
        ec_run_stats
    """)

run_stats.head()


In [0]:
#@title Viz: Times taken per run where time_taken > 0
alt.Chart(run_stats.loc[run_stats['time_taken'] > 0], 
          title='Time taken to complete run').mark_boxplot(
    extent='min-max', size=20).encode(
    x='overall_run_no:O',
    y='time_taken:Q',
    color='week_no:N'
).properties(width=1000, height=500).configure_axis(
    grid=False
)

In [0]:
# median time taken where time taken > 0
median = run_stats.loc[run_stats['time_taken'] > 0].groupby(
    ['overall_run_no', 'week_no'],
    as_index=False)['time_taken'].median()

median.head()


In [0]:
#@title Viz: Median time taken to complete run where time_taken > 0
alt.Chart(median,
          title='Median time taken to complete run where time_taken > 0').mark_bar(
    size=20).encode(
    x='overall_run_no',
    y='time_taken',
    color='week_no:N'
).properties(width=1000, height=500)


In [0]:
# mean time taken
mean = run_stats.groupby(
    ['overall_run_no', 'week_no'],
    as_index=False)['time_taken'].mean()

mean.head()


In [0]:
#@title Viz: Mean time taken to complete run (including t=0)
alt.Chart(mean,
          title='Mean time taken to complete run').mark_bar(
    size=20).encode(
    x='overall_run_no',
    y='time_taken',
    color='week_no:N'
).properties(width=1000, height=500)


# Completion times per run - cohort 

In [0]:
run_stats_c = sql_to_df("""
    SELECT
        overall_run_no,
        time_taken,
        week_no
    FROM
        ec_run_stats
    WHERE
        DATE(first_completed) BETWEEN '2019-05-06' AND '2019-05-12'
    """)

run_stats_c.head()


In [0]:
#@title Viz: Cohort - Time taken per run where time_taken > 0
alt.Chart(run_stats_c.loc[run_stats_c['time_taken'] > 0], 
          title='Cohort - Time taken to complete run where t > 0').mark_boxplot(
    extent='min-max', size=20).encode(
    x='overall_run_no:O',
    y='time_taken:Q',
    color='week_no:N'
).properties(width=1000, height=500).configure_axis(
    grid=False
)

In [0]:
# median time taken where time taken > 0
median_c = run_stats_c.loc[run_stats_c['time_taken'] > 0].groupby(
    ['overall_run_no', 'week_no'],
    as_index=False)['time_taken'].median()

median_c.head()


In [0]:
#@title Viz: Cohort - Median time taken to complete run where time_taken > 0
alt.Chart(median_c,
          title='Cohort - Median time taken to complete run where time_taken > 0').mark_bar(
    size=20).encode(
    x='overall_run_no',
    y='time_taken',
    color='week_no:N'
).properties(width=1000, height=500)


In [0]:
# mean time taken cohort
mean_c = run_stats_c.groupby(
    ['overall_run_no', 'week_no'],
    as_index=False)['time_taken'].mean()

mean_c.head()


In [0]:
#@title Viz: Cohort - Mean time taken to complete run (including t=0)
alt.Chart(mean_c,
          title='Cohort - Mean time taken to complete run').mark_bar(
    size=20).encode(
    x='overall_run_no',
    y='time_taken',
    color='week_no:N'
).properties(width=1000, height=500)


# Completion times of the programme

In [0]:
df = sql_to_df("""
    SELECT
        device_id,
        overall_run_no,
        first_completed
    FROM
        ec_run_stats""")

df.head()

In [0]:
# create dataframes for first and last run completed
first = df.loc[df['overall_run_no'] == 1]
last = df.loc[df['overall_run_no'] == 27]

# inner join the two dataframes
result = first.merge(
    last,
    on='device_id',
    how='inner',
    suffixes=('_1', '_27'))

result.head()


In [0]:
# sense checking by checking random user from 'result'
df.loc[df['device_id'] == '006253AB-CE4A-4533-919E-71807FC21E7C']


In [0]:
# use dt.days to convert timedeltas to float
result['timedelta'] = (
    result['first_completed_27'] - result['first_completed_1']
).dt.days

result.head()


In [0]:
# remove NaN
result.dropna(axis=0, inplace=True)

result.head()


In [0]:
# data cleaning as some deltas are negative
# create a mask for plot slice
clean = result['timedelta'] > 0


In [0]:
#@title Viz: Days between first and last run
result.loc[clean].boxplot(
    column='timedelta', 
    figsize=(12,10)
);


In [0]:
# final dataframe to replicate in SQL
result.loc[clean].filter(
    items=[
        'device_id',
        'first_completed_1',
        'first_completed_27',
        'timedelta'
    ]
).head()
